In [ ]:
import pandas as pd
from visualize import plot_words_with_colors
import ast

In [ ]:
df = pd.read_csv('/calderon-gender-prediction/results/scenes_word_attributions.csv')
df.head()

I want to zip the dataframe containting the play / character information with the attributions dataframe.

In [ ]:
scenes_df = pd.read_csv('/calderon-gender-prediction/results/scenes_bert-base-spanish_1e-05_32_7.csv')
scenes_df.head()

In [ ]:
#zip the two dataframes together
new_df = pd.concat([scenes_df, df], axis=1)
new_df.head()

In [ ]:
masked_scenes_attributions = pd.read_csv('/calderon-gender-prediction/results/scenes_word_attributions.csv')
masked_scenes_predictions = pd.read_csv('/calderon-gender-prediction/results/scenes_bert-base-spanish_1e-05_32_7.csv')

masked_scenes_df = pd.concat([masked_scenes_predictions, masked_scenes_attributions], axis=1)

# 1. Find the most positive and most negative **tokens** based on their attributions

In [ ]:
masked_tokens = pd.read_csv('/calderon-gender-prediction/results/tokens_bert-base-spanish_1e-05_24_5.csv')
masked_tokens_attributions = pd.read_csv('/calderon-gender-prediction/results/tokens_word_attributions.csv')

masked_df = pd.concat([masked_tokens, masked_tokens_attributions], axis=1)
masked_df.head()

In [ ]:
attributions = get_word_attributions(masked_tokens_attributions)

In [ ]:
#get average attribution value for each word
word_attributions_dict = {}
word_counts = {}
for word, attribution in attributions:
    if word in word_attributions_dict:
        word_attributions_dict[word] += attribution
        word_counts[word] += 1
    else:
        word_attributions_dict[word] = attribution
        word_counts[word] = 1

word_attributions_avg = {word: word_attributions_dict[word] / word_counts[word] for word in word_attributions_dict}

#sort the words by average attribution value
word_attributions_avg = {k: v for k, v in sorted(word_attributions_avg.items(), key=lambda item: item[1])}


print('Top 20 most negative average attributions:')
fem_words = list(word_attributions_avg.items())[:40]
# fem_words

print('Top 20 most positive average attributions:')
masc_words = list(word_attributions_avg.items())[-40:]
# masc_words

In [ ]:
fem_words

In [ ]:
masc_words

I want to find every instance of a polarized word that has been run through the interpretor model. I'll do it by looking at the attributions, so, I'll take a word attribution pair, and find it in the column all_attributions, then run that column through my visualization function

In [ ]:
df = pd.read_csv('/calderon-gender-prediction/results/masked_tokens_df.csv')

In [ ]:
def findcontext(attribution, df):
    for index, row in df.iterrows():
        #read the column all_attributions and convert it to a list
        attributions = ast.literal_eval(row['all_attributions'])
        attributions = attributions[1:-1]
        attribtions = [ast.literal_eval(x) for x in attributions]
        tuples_list = [ast.literal_eval(item) for item in attributions]

        
        if any(attribution[0] == x[0] for x in tuples_list):
            print(row['character_id'], row['id'],row['is_male'], row['predictions'], row['probabilities'])
            plot_words_with_colors(tuples_list)

In [ ]:
for attribution in fem_words:
    print(attribution[0])

    plot = findcontext(attribution, df)